# observations_refit_csv

## setup

In [56]:
from lxml import etree
import os
import pandas as pd
import csvw_functions
import json
from rdflib import Graph, SOSA, URIRef
import csv

## Read in the data

In [57]:
data_dir=r'C:\Users\cvskf\OneDrive - Loughborough University\_Data\REFIT_Smart_Home_dataset_PUBLIC_v1\Data'

In [58]:
tree=etree.parse(os.path.join(data_dir,'REFIT_BUILDING_SURVEY.xml'))
NS={'refit':'http://www.refitsmarthomes.org'}
tree

## Create csv

### header line

In [59]:
with open('observation_survey_data.csv','w', newline='') as f1:
    csvwriter=csv.writer(f1)
    csvwriter.writerow(['id','type','madeBySensor','usedProcedure','hasFeatureOfInterest',
                                    'observedProperty','hasSimpleResult','hasResult','value','unit','resultTime'])

In [60]:
i=25312398

### Add buildings

In [61]:
building_nodes=tree.xpath('//refit:Building', namespaces=NS)
len(building_nodes)

20

In [62]:
with open('observation_survey_data.csv','a', newline='') as f1:
    csvwriter=csv.writer(f1)
    
    for building_node in building_nodes:
        
        for k,v in building_node.attrib.items():
            
            if k in ['id','startDateTime']:
            
                continue
            
            elif k=='orientation':
                
                csvwriter.writerow([f'Observation{str(i)}','Observation','Researcher','SiteVisit',building_node.get('id'),
                                    k,'','',v,'DEG',building_node.get('startDateTime')])
                
            elif k=='loftInsulationThickness':
                
                if v.endswith('mm'):
                    
                    csvwriter.writerow([f'Observation{str(i)}','Observation','Researcher','SiteVisit',building_node.get('id'),
                                    k,'','',v.removesuffix('mm'),'MilliM',building_node.get('startDateTime')])
                    
                else:
                    
                    raise Exception
                
                
            else:
                
                csvwriter.writerow([f'Observation{str(i)}','Observation','Researcher','SiteVisit',building_node.get('id'),
                                    k,'',v.replace(' ','_'),'','',building_node.get('startDateTime')])
                
            i+=1
                

### Spaces

In [63]:
space_nodes=tree.xpath('//refit:Space', namespaces=NS)
len(space_nodes)

389

In [64]:
with open('observation_survey_data.csv','a', newline='') as f1:
    csvwriter=csv.writer(f1)

    for space_node in space_nodes:

        building_node=space_node.getparent()
        
        csvwriter.writerow([f'Observation{str(i)}','Observation','Researcher','SiteVisit',space_node.get('id'),
                            'hasBuilding','',building_node.get('id'),'','',space_node.get('startDateTime')])
    
        i+=1
    

In [65]:
with open('observation_survey_data.csv','a', newline='') as f1:
    csvwriter=csv.writer(f1)
    
    for space_node in space_nodes:
        
        for k,v in space_node.attrib.items():
            
            if k in ['id','startDateTime']:
            
                continue
            
            elif k=='area':
                
                csvwriter.writerow([f'Observation{str(i)}','Observation','Researcher','SiteVisit',space_node.get('id'),
                                    k,'','',v,'M2',space_node.get('startDateTime')])
                
            elif k=='volume':
                
                csvwriter.writerow([f'Observation{str(i)}','Observation','Researcher','SiteVisit',space_node.get('id'),
                                    k,'','',v,'M3',space_node.get('startDateTime')])
                
            elif k=='storeyLevel':
                
                csvwriter.writerow([f'Observation{str(i)}','Observation','Researcher','SiteVisit',space_node.get('id'),
                                    k,'',f'Storey{v}','','',space_node.get('startDateTime')])
                
            else:
                
                csvwriter.writerow([f'Observation{str(i)}','Observation','Researcher','SiteVisit',space_node.get('id'),
                                    k,'',v.replace(' ','_'),'','',space_node.get('startDateTime')])
                
            i+=1
                

## Testing

In [66]:
df=pd.read_csv('observation_survey_data.csv')
df

,id,type,madeBySensor,usedProcedure,hasFeatureOfInterest,observedProperty,hasSimpleResult,hasResult,value,unit,resultTime
0,Observation25312398,Observation,Researcher,SiteVisit,Building01,occupancyType,NaN,Single_family_dwelling,NaN,NaN,2013-10-01T00:00:00Z
1,Observation25312399,Observation,Researcher,SiteVisit,Building01,builtFormType,NaN,Detached_house_or_bungalow,NaN,NaN,2013-10-01T00:00:00Z
2,Observation25312400,Observation,Researcher,SiteVisit,Building01,orientation,NaN,NaN,327,DEG,2013-10-01T00:00:00Z
3,Observation25312401,Observation,Researcher,SiteVisit,Building01,wallTypeMainBuilding,NaN,Masonry-Boxwall-Cavity,NaN,NaN,2013-10-01T00:00:00Z
4,Observation25312402,Observation,Researcher,SiteVisit,Building01,wallAgeBandMainBuilding,NaN,1975_-_1980,NaN,NaN,2013-10-01T00:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...
2576,Observation25314974,Observation,Researcher,SiteVisit,Space386,conditionType,NaN,Not_conditioned,NaN,NaN,2014-02-07T00:00:00Z
2577,Observation25314975,Observation,Researcher,SiteVisit,Space386,area,NaN,NaN,43.68,M2,2014-02-07T00:00:00Z
2578,Observation25314976,Observation,Researcher,SiteVisit,Space386,volume,NaN,NaN,42.819091,M3,2014-02-07T00:00:00Z
2579,Observation25314977,Observation,Researcher,SiteVisit,Space386,storeyLevel,NaN,Storey2,NaN,NaN,2014-02-07T00:00:00Z


### spaces in building01

In [70]:
df[(df.observedProperty=='hasBuilding') & (df.hasResult=='Building01')].hasFeatureOfInterest.values

array(['Space1', 'Space2', 'Space3', 'Space4', 'Space5', 'Space6',
       'Space7', 'Space8', 'Space9', 'Space10', 'Space11', 'Space12',
       'Space13', 'Space14', 'Space15', 'Space16', 'Space17', 'Space18',
       'Space325', 'Space346'], dtype=object)